In [1]:
import pandas as pd 
import numpy as np 
import time
import pulp

In [2]:
solver = pulp.get_solver('GUROBI')

In [3]:
ordem_operacoes=pd.read_excel("Dados_DFJSP.xlsx", sheet_name="ordem_operacoes")
tempo_de_processamento=pd.read_excel("Dados_DFJSP.xlsx", sheet_name="tempo_de_processamento")

# ordem_operacoes.replace({0:None}, inplace = True)
# tempo_de_processamento.replace({0:None}, inplace = True)

ops = ordem_operacoes.values
pt = tempo_de_processamento.values

In [4]:
# número de jobs
n_jobs = len(ordem_operacoes["Job"].unique())

# numero de operações
n_operations = len(tempo_de_processamento["Operacao"].unique())

# numero de job
n_maquinas = len(tempo_de_processamento["maquina"].unique())

In [5]:
print(f"Número de jobs: ", n_jobs)
print(f"Número de operações: ", n_operations)
print(f"Número de maquinas: ", n_maquinas)

Número de jobs:  5
Número de operações:  6
Número de maquinas:  5


In [6]:
Cmax = pulp.LpVariable('Cmax',lowBound = 0, cat='Continuous')

Ci = pulp.LpVariable.dicts("start_time",
                           ((i) for i in range(1,n_jobs + 1)),
                           lowBound=0,
                           cat='Continuous')
#i=job,j=operation,k=maquina
st = pulp.LpVariable.dicts("start_time",
                           ((i, j, k) for i in range(1,n_jobs + 1) for j in [ops[i-1][o] for o in range(1,n_operations - 1) if ops[i-1][o]!=0] for k in list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==j),'maquina'])),
                           lowBound=0,
                           cat='Continuous')


In [7]:

ct = pulp.LpVariable.dicts("completed_time",
                           ((i, j, k) for i in range(1,n_jobs + 1) for j in [ops[i-1][o] for o in range(1,n_operations - 1) if ops[i-1][o]!=0] for k in list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==j),'maquina'])),
                           lowBound=0,
                           cat='Continuous')

x = pulp.LpVariable.dicts("X_binary_var",
                          ((i, j, k) for i in range(1,n_jobs + 1) for j in [ops[i-1][o] for o in range(1,n_operations - 1) if ops[i-1][o]!=0] for k in list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==j),'maquina'])),
                          lowBound=0, upBound=1,
                          cat='Binary')

y = pulp.LpVariable.dicts("Y_binary_var",
                          ((i, j, l, m,k) for i in range(1,n_jobs + 1) for j in [ops[i-1][o] for o in range(1,n_operations - 1) if ops[i-1][o]!=0]  for l in range(1,6) for m in [ops[l-1][a] for a in range(1,5) if ops[l-1][a]!=0]  for k in list(set(list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==j),'maquina']))&set(list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==m),'maquina']))) if i < l ),
                          lowBound=0, upBound=1,
                          cat='Binary')

In [8]:
model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)
model += Cmax

In [9]:
for i in range(1,n_jobs + 1):
    for j in [ops[i-1][o] for o in range(1,n_operations - 1) if ops[i-1][o]!=0]:
        ma_list=list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==j),'maquina'])
        model += pulp.lpSum(x[i,j,ma] for ma in ma_list)==1

In [10]:
for i in range(1,n_jobs + 1):
    for j in [ops[i-1][o] for o in range(1,n_operations - 1) if ops[i-1][o]!=0]:
        ma_list=list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==j),'maquina'])
        for k in ma_list:
            ptime=int(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==j)&(tempo_de_processamento['maquina']==k),[f"Job{i}"]].iloc[0])
            I = i
            J = j
            K = k
            model += st[I,J,K] + ct[I,J,K] <=x[I,J,K]*1000
            model += ct[I,J,K] >= st[I,J,K] +ptime-(1-x[I,J,K])*1000

In [11]:
for i in range(1,n_jobs + 1):
    for l in range(1,n_jobs + 1):
        if i<l:
            for o1 in [ops[i-1][o] for o in range(1,n_operations - 1) if ops[i-1][o]!=0]:
                ma1_list=list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==o1)&(tempo_de_processamento.loc[:][f"Job{i}"]!=0),'maquina'])
                for o2 in [ops[l-1][o] for o in range(1,n_operations - 1) if ops[l-1][o]!=0]:
                    ma2_list=list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==o2)&(tempo_de_processamento.loc[:][f"Job{l}"]!=0),'maquina'])
                    mach_set=list(set(ma1_list)&set(ma2_list))
                    if len(mach_set)>0:
                        for k in mach_set:
                            model += st[i,o1,k]>=ct[l,o2,k]-y[i,o1,l,o2,k]*1000
                            model += st[l,o2,k]>=ct[i,o1,k]-(1-y[i,o1,l,o2,k])*1000

In [12]:
for i in range(1,n_jobs + 1):
    op_num=[ops[i-1][o] for o in range(1,n_operations - 1) if ops[i-1][o]!=0]
    print(op_num)
    for j in range(1,len(op_num)):
        print(i,j)
        preop_num=op_num[j-1]
        curop_num=op_num[j]
        print("J%d"%i)
        print('pc')
        print(preop_num,curop_num)
        prema_list=list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==preop_num)&(tempo_de_processamento.loc[:][f"Job{i}"]!=0),'maquina'])
        curma_list=list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==curop_num)&(tempo_de_processamento.loc[:][f"Job{i}"]!=0),'maquina'])
        model += pulp.lpSum(st[i,curop_num,ma1] for ma1 in curma_list) >= pulp.lpSum(ct[i,preop_num,ma2] for ma2 in prema_list)

[5, 4, 2, 3]
1 1
J1
pc
5 4
1 2
J1
pc
4 2
1 3
J1
pc
2 3
[1, 4, 2]
2 1
J2
pc
1 4
2 2
J2
pc
4 2
[3, 5, 1, 4]
3 1
J3
pc
3 5
3 2
J3
pc
5 1
3 3
J3
pc
1 4
[3, 2, 5, 4]
4 1
J4
pc
3 2
4 2
J4
pc
2 5
4 3
J4
pc
5 4
[4, 6]
5 1
J5
pc
4 6


In [13]:
for i in range(1,n_jobs + 1):
    op_list=[ops[i-1][o] for o in range(1,n_operations - 1) if ops[i-1][o]!=0]
    last_op=op_list[-1]
    model +=Ci[i]>=pulp.lpSum(ct[i,last_op,k] for k in list(tempo_de_processamento.loc[(tempo_de_processamento['Operacao']==last_op),'maquina']))
    model += Cmax >= Ci[i]

In [14]:
model.solve(solver)
pulp.LpStatus[model.status]     

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 337 rows, 223 columns and 1001 nonzeros
Model fingerprint: 0x9be162e4
Variable types: 74 continuous, 149 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 23 rows and 23 columns
Presolve time: 0.00s
Presolved: 314 rows, 200 columns, 921 nonzeros
Variable types: 68 continuous, 132 integer (132 binary)
Found heuristic solution: objective 449.0000000

Root relaxation: objective 1.650000e+02, 143 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  165.00000    0   21  449

'Optimal'

In [15]:
for var in ct:
    var_value = ct[var].varValue
    print( var[0],var[1],var[2] ,var_value)

total_cost = pulp.value(model.objective)
print ('min cost:',total_cost)

1 5 2 18.0
1 5 3 0.0
1 4 4 0.0
1 4 2 148.0
1 2 2 188.0
1 2 3 0.0
1 3 5 193.0
1 3 4 0.0
2 1 3 75.0
2 1 5 0.0
2 4 4 168.0
2 4 2 0.0
2 2 2 0.0
2 2 3 193.0
3 3 5 18.0
3 3 4 0.0
3 5 2 74.0
3 5 3 0.0
3 1 3 0.0
3 1 5 179.0
3 4 4 193.0
3 4 2 0.0
4 3 5 58.0
4 3 4 7.105427357601002e-15
4 2 2 0.0
4 2 3 88.0
4 5 2 0.0
4 5 3 168.0
4 4 4 179.0
4 4 2 0.0
5 4 4 81.0
5 4 2 0.0
5 6 2 0.0
5 6 1 169.0
min cost: 193.0


In [16]:
for i in x:
    print(i[0],i[1],i[2],'----',x[i].varValue)

1 5 2 ---- 1.0
1 5 3 ---- 0.0
1 4 4 ---- 0.0
1 4 2 ---- 1.0
1 2 2 ---- 1.0
1 2 3 ---- 0.0
1 3 5 ---- 1.0
1 3 4 ---- -0.0
2 1 3 ---- 1.0
2 1 5 ---- 0.0
2 4 4 ---- 1.0
2 4 2 ---- -0.0
2 2 2 ---- -0.0
2 2 3 ---- 1.0
3 3 5 ---- 1.0
3 3 4 ---- 0.0
3 5 2 ---- 1.0
3 5 3 ---- 0.0
3 1 3 ---- -0.0
3 1 5 ---- 1.0
3 4 4 ---- 1.0
3 4 2 ---- -0.0
4 3 5 ---- 1.0
4 3 4 ---- -0.0
4 2 2 ---- -0.0
4 2 3 ---- 1.0
4 5 2 ---- -0.0
4 5 3 ---- 1.0
4 4 4 ---- 1.0
4 4 2 ---- -0.0
5 4 4 ---- 1.0
5 4 2 ---- 0.0
5 6 2 ---- 0.0
5 6 1 ---- 1.0


In [23]:
148 %60

28

In [17]:
import plotly.figure_factory as ff 

In [24]:
df = [
    dict(Task="Maquina 2", Start='2022-12-11 12:00:00', Finish='2022-12-11 12:18:00', Resource = "Job 1"),
    dict(Task="Maquina 3", Start='2022-12-11 12:00:00', Finish='2022-12-11 12:00:00', Resource = "Job 1"),
    dict(Task="Maquina 4", Start='2022-12-11 12:00:00', Finish='2022-12-11 12:00:00', Resource = "Job 1"),
    dict(Task="Maquina 2", Start='2022-12-11 12:00:00', Finish='2022-12-11 14:28:00', Resource = "Job 1"),
    ]
fig = ff.create_gantt(df)
fig.show()

In [33]:
import pandas as pd
import plotly.figure_factory as ff
import datetime
from operator import itemgetter


m_keys=[j+1 for j in range(6)]
j_keys=[j for j in range(5)]
index_record=[]
j_record={}

for var in x:
    if x[var].varValue==1:
        start_time=str(datetime.timedelta(seconds=st[var[0],var[1],var[2]].varValue)) # convert seconds to hours, minutes and seconds
        end_time=str(datetime.timedelta(seconds=ct[var[0],var[1],var[2]].varValue))
        j_record[(var[0],var[1],var[2])]=[start_time,end_time]
        index_record.append([var[0],var[1],var[2]])
index_record.sort(key=itemgetter(2))
        

df=[]
for i in index_record:
    df.append(dict(Task='Maquina %s'%(i[2]), Start='2022-12-11 %s'%(str(j_record[(i[0],i[1],i[2])][0])), Finish='2022-12-11 %s'%(str(j_record[(i[0],i[1],i[2])][1])),Resource='Job %s'%(i[0])))
    
fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True, title='Agendamento das Prodruções de linguiças')
# fig.add_vline(x="2022-12-11 03:13:00")
fig

In [44]:
import pandas as pd
import plotly.figure_factory as ff
import datetime
from operator import itemgetter


m_keys=[j+1 for j in range(6)]
j_keys=[j for j in range(5)]
index_record=[]
j_record={}

for var in x:
    if x[var].varValue==1:
        start_time=str(datetime.timedelta(seconds=st[var[0],var[1],var[2]].varValue)) # convert seconds to hours, minutes and seconds
        end_time=str(datetime.timedelta(seconds=ct[var[0],var[1],var[2]].varValue))
        j_record[(var[0],var[1],var[2])]=[start_time,end_time]
        index_record.append([var[0],var[1],var[2]])
index_record.sort(key=itemgetter(2))
        

df=[]
for i in index_record:
    df.append(dict(Task='Maquina %s'%(i[2]), Start='2022-12-11 %s'%(str(j_record[(i[0],i[1],i[2])][0])), Finish='2022-12-11 %s'%(str(j_record[(i[0],i[1],i[2])][1])),Resource='Job %s'%(i[0])))
    
fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True, title='Agendamento das Prodruções de linguiças')
# fig.add_vline(x="2022-12-11 00:03:13", annotation_text="decline", annotation_position="top left")
fig.add_vrect(x0="2022-12-11 00:03:13", x1="2022-12-11 00:03:15", row="all", col=1, annotation_text="Cmax = 193 min", annotation_position="top left", fillcolor="violet", opacity=0.50, line_width=0, annotation=dict(font_size=30, font_family="Times New Roman"))
fig